<a href="https://colab.research.google.com/github/totiela/vk-ml-time-series-test-task/blob/main/notebooks/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек и загрузка данных

Для получения идентичных предсказаний, необходимо запускать ноутбук на ***видеокарте***.

In [1]:
# Установка необходимых библиотек
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.0 MB/s eta 0:00:00


In [9]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import joblib
import os
from catboost import Pool, CatBoostClassifier
from xgboost import XGBClassifier
import warnings

# Отключение всех предупреждений
warnings.filterwarnings('ignore')

In [10]:
# Клонируем репозитоиий
!git clone https://github.com/totiela/vk-ml-time-series-test-task.git
%cd vk-ml-time-series-test-task

Cloning into 'vk-ml-time-series-test-task'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 117 (delta 17), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 47.79 MiB | 9.45 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/vk-ml-time-series-test-task/vk-ml-time-series-test-task


In [11]:
# Функция для проверки существования файла
def check_file_exists(filepath):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Файл не найден: {filepath}")

# Пути к данным и моделям
test_data_path = 'data/raw/test.parquet'
test_116_features_path = 'data/processed/test/test_116_features.parquet'
test_141_features_path = 'data/processed/test/test_141_features.parquet'

cb_tuned_116_path = 'models/cb_tuned_116.pkl'
xgb_tuned_116_path = 'models/xgb_tuned_116.pkl'
cb_no_tuned_141_path = 'models/cb_no_tuned_141.pkl'
xgb_no_tuned_141_path = 'models/xgb_no_tuned_141.pkl'

# Проверка файлов и загрузка данных
check_file_exists(test_data_path)
test_df = pd.read_parquet(test_data_path)

check_file_exists(test_116_features_path)
test_116_features = pd.read_parquet(test_116_features_path)

check_file_exists(test_141_features_path)
test_141_features = pd.read_parquet(test_141_features_path)

# Проверка моделей и их загрузка
check_file_exists(cb_tuned_116_path)
cb_tuned_116 = joblib.load(cb_tuned_116_path)

check_file_exists(xgb_tuned_116_path)
xgb_tuned_116 = joblib.load(xgb_tuned_116_path)

check_file_exists(cb_no_tuned_141_path)
cb_no_tuned_141 = joblib.load(cb_no_tuned_141_path)

check_file_exists(xgb_no_tuned_141_path)
xgb_no_tuned_141 = joblib.load(xgb_no_tuned_141_path)

print("Данные и модели успешно загружены.")

Данные и модели успешно загружены.


# Получение предсказаний

In [12]:
# Функция для предстказаний
def blended_predict(X_116, X_141):
    """
    Получает вероятностные предсказания от четырех моделей и возвращает взвешенное блендированное предсказание.

    Параметры:
    - X_116 (pd.DataFrame): Набор признаков для моделей, обученных на 116 признаках.
    - X_141 (pd.DataFrame): Набор признаков для моделей, обученных на 141 признаке.

    Возвращает:
    - blended_pred (np.ndarray): Взвешенные вероятности для класса 1 после блендирования всех четырех моделей.
    """

    # Получение предсказаний вероятностей от модели CatBoost, настроенной для 116 признаков
    cb_tuned_116_pred = cb_tuned_116.predict_proba(X_116)[:, 1]

    # Получение предсказаний вероятностей от модели XGBoost, настроенной для 116 признаков
    xgb_tuned_116_pred = xgb_tuned_116.predict_proba(X_116)[:, 1]

    # Получение предсказаний вероятностей от модели CatBoost без настройки для 141 признака
    cb_no_tuned_141_pred = cb_no_tuned_141.predict_proba(X_141)[:, 1]

    # Получение предсказаний вероятностей от модели XGBoost без настройки для 141 признака
    xgb_no_tuned_141_pred = xgb_no_tuned_141.predict_proba(X_141)[:, 1]

    # Взвешенное блендирование предсказаний от всех четырех моделей
    blended_pred = (
        0.3 * cb_tuned_116_pred +
        0.3 * xgb_tuned_116_pred +
        0.3 * cb_no_tuned_141_pred +
        0.1 * xgb_no_tuned_141_pred
    )

    # Возврат итогового блендированного предсказания
    return blended_pred

In [13]:
# Вызов функции и получение предсказаний
predictions = blended_predict(test_116_features, test_141_features)

In [14]:
# Сохранение полученных предсказаний
from google.colab import files

submission_df = pd.DataFrame(
    {
        'id': test_df['id'],
        'score': predictions
    }
)

# Сохранение DataFrame в CSV файл
submission_df.to_csv('submission.csv', index=False)
# Скачивание файла
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>